# The Battle Of Neighbourhoods


# Introduction 

Opening a new shopping mall in Hyderabad,India


In [1]:
!pip install geocoder
!pip install geopy
!pip install bs4
!pip install scikit-learn
!pip install folium

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


# Getting the Data

Build a dataframe of neighborhoods in Hyderabad, India by web scraping the data from Wikipedia page
Get the geographical coordinates of the neighborhoods by Python Geocoder package
Obtain the venue data for the neighborhoods from Foursquare API
Explore and cluster the neighbourhoods
Select the best cluster to open a new shopping mall

# Business Problem

This project is mainly focused on geospatial analysis of the Hyderabad City to understand which would be the best place to open a new mall

In [3]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India").text

In [4]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [5]:
# create a list to store neighborhood data
neighborhoodList = []

In [6]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [7]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,A. C. Guards
1,A. S. Rao Nagar
2,Abhyudaya Nagar
3,Abids
4,Adibatla


In [8]:
# print the number of rows of the dataframe
kl_df.shape

(200, 1)

In [9]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Hyderabad, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [11]:
coords

[[17.395014876941943, 78.45981230832962],
 [17.411200000000065, 78.50824000000006],
 [17.337650000000053, 78.56414000000007],
 [17.389800000000037, 78.47658000000007],
 [17.235790000000065, 78.54130000000004],
 [17.410610000000077, 78.51513000000006],
 [17.37751000000003, 78.48005000000006],
 [17.389149685816093, 78.46385388458206],
 [17.34259000000003, 78.47626000000008],
 [17.36068000000006, 78.47998000000007],
 [17.503370000000075, 78.41602000000006],
 [17.535430000000076, 78.54427000000004],
 [17.385820000000024, 78.51836000000003],
 [17.53332000000006, 78.32529000000005],
 [17.435350000000028, 78.44861000000003],
 [17.45787000000007, 78.53882000000004],
 [17.40784000000002, 78.49150000000003],
 [17.385140000000035, 78.44738000000007],
 [17.369170000000054, 78.43683000000004],
 [17.40710000000007, 78.50233000000003],
 [17.372720000000072, 78.49047000000007],
 [17.388960000000054, 78.48681000000005],
 [17.39931000000007, 78.49964000000006],
 [17.339920000000063, 78.54553000000004],


In [12]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [13]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [14]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df.head()

(200, 3)


,Neighborhood,Latitude,Longitude
0,A. C. Guards,17.395015,78.459812
1,A. S. Rao Nagar,17.411200,78.508240
2,Abhyudaya Nagar,17.337650,78.564140
3,Abids,17.389800,78.476580
4,Adibatla,17.235790,78.541300


In [16]:
# get the coordinates of Hyderabad
address = 'Hyderabad, India'

geolocator = Nominatim(user_agent="user_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hyderabad, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hyderabad, India 17.38878595, 78.46106473453146.


In [17]:
# create map of Hyderabad using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [19]:
# save the map as HTML file
map_kl.save('./map_kl.html')

# Use the foursquare API to explore the neighbourhoods

In [20]:
CLIENT_ID = 'RJ3CLZG4JCDCVREY5S2R1WFPNX0A03MYJRD4CTPAWWBQJW2K' # your Foursquare ID
CLIENT_SECRET = 'TJ4NPL40BJWU4A51PSHNXVK2ADD2FZRPANQWRNONGGDAMCYN' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RJ3CLZG4JCDCVREY5S2R1WFPNX0A03MYJRD4CTPAWWBQJW2K
CLIENT_SECRET:TJ4NPL40BJWU4A51PSHNXVK2ADD2FZRPANQWRNONGGDAMCYN


In [21]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [22]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(10)

(6322, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,A. C. Guards,17.395015,78.459812,Cafe Niloufer & Bakers,17.399715,78.462881,Café
1,A. C. Guards,17.395015,78.459812,Subhan Bakery,17.392412,78.464712,Bakery
2,A. C. Guards,17.395015,78.459812,Chicha's,17.403255,78.460152,Hyderabadi Restaurant
3,A. C. Guards,17.395015,78.459812,Taiba Bakers & Confectioners,17.402530,78.456823,Bakery
4,A. C. Guards,17.395015,78.459812,Nizam club,17.403221,78.468729,Lounge
5,A. C. Guards,17.395015,78.459812,Rayalaseema Ruchulu,17.403084,78.463012,South Indian Restaurant
6,A. C. Guards,17.395015,78.459812,DineHill,17.405256,78.451674,Indian Restaurant
7,A. C. Guards,17.395015,78.459812,Prince Hotel,17.394736,78.442410,Indian Restaurant
8,A. C. Guards,17.395015,78.459812,Spice 6,17.409007,78.450559,Bistro
9,A. C. Guards,17.395015,78.459812,Siraj's International Juice Center,17.395579,78.442701,Juice Bar


# Lets check how many venues were returned for each neighbourhood

In [23]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
A. C. Guards,73,73,73,73,73,73
A. S. Rao Nagar,20,20,20,20,20,20
Abhyudaya Nagar,9,9,9,9,9,9
Abids,81,81,81,81,81,81
Adikmet,18,18,18,18,18,18
Afzal Gunj,46,46,46,46,46,46
Aghapura,64,64,64,64,64,64
"Aliabad, Hyderabad",9,9,9,9,9,9
Alijah Kotla,15,15,15,15,15,15


In [24]:
#venues
#venues_df[venues_df['Neighborhood']=='Sikh Village'].groupby

# Let's find out how many unique categories can be curated from all the returned venues

In [25]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 180 uniques categories.


In [26]:
venues_df['VenueCategory'].unique()[:50] #displays all the 174 category names

array(['Café', 'Bakery', 'Hyderabadi Restaurant', 'Lounge',
       'South Indian Restaurant', 'Indian Restaurant', 'Bistro',
       'Juice Bar', 'Science Museum', 'Middle Eastern Restaurant',
       'Ice Cream Shop', 'Park', 'Hotel', 'Snack Place',
       'Vegetarian / Vegan Restaurant', 'Shoe Store', 'Hotel Bar',
       'Diner', 'Stadium', 'Neighborhood', 'Dessert Shop',
       'Performing Arts Venue', 'Fast Food Restaurant',
       'Mobile Phone Shop', 'Chinese Restaurant', 'Pizza Place',
       'Coffee Shop', 'Department Store', 'Shopping Mall',
       'Clothing Store', 'Fried Chicken Joint', 'Electronics Store',
       'Hookah Bar', 'Bookstore', 'Planetarium', 'Music Store',
       'Movie Theater', 'Sandwich Place', 'Convenience Store',
       'Asian Restaurant', 'Pharmacy', 'Restaurant', 'Food Truck',
       'Chaat Place', 'Burger Joint', 'Food', 'Smoke Shop', 'Multiplex',
       'Breakfast Spot', 'Bar'], dtype=object)

In [27]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

# Analyze each neighbourhood

In [28]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head(20)

(6322, 181)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport Service,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Workshop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Business Service,Cable Car,Cafeteria,Café,Campground,Candy Store,Caribbean Restaurant,Castle,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Irish Pub,Italian Restaurant,Juice Bar,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Night Market,Nightclub,Nightlife Spot,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Plaza,Pool,Pool Hall,Pub,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Taco Place,Tea Room,Temple,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wings Joint,Women's Store,Zoo
0,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [29]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).sum().reset_index()
#df.group/by("state")["last_name"].count()

#kl_onehot.head()
print(kl_grouped.shape)
kl_grouped

(198, 181)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport Service,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Workshop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Business Service,Cable Car,Cafeteria,Café,Campground,Candy Store,Caribbean Restaurant,Castle,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Irish Pub,Italian Restaurant,Juice Bar,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Night Market,Nightclub,Nightlife Spot,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Plaza,Pool,Pool Hall,Pub,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Taco Place,Tea Room,Temple,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wings Joint,Women's Store,Zoo
0,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,2,2,1,0,0,1,0,0,0,0,5,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,7,2,0,0,2,3,13,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,1,0,1,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
1,A. S. Rao Nagar,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Abhyudaya Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Abids,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,1,0,2,0,0,1,2,0,0,0,3,0,0,0,0,1,4,1,0,2,0,0,0,0,0,0,0,2,3,1,0,0,0,0,0,0,1,4,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,5,1,0,0,0,5,10,0,1,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,2,1,0,0,0,0,0,0,0,0,0,3,0,0,0,0,1,1,0,1,1,2,0,0,2,0

In [30]:
#a=kl_grouped['Neighborhoods'][0]


In [31]:
len((kl_grouped[kl_grouped["Shopping Mall"] > 0]))  #So there are 79 shopping malls in hyderabad(which it very high)
#So now we want to select a good location where the no of shopping malls are less so that our chances of setting up a shopping mall at that location  should be good

62

# Create a dataframe for shopping mall data only

In [32]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [33]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,A. C. Guards,1
1,A. S. Rao Nagar,0
2,Abhyudaya Nagar,0
3,Abids,1
4,Adikmet,0


# Now cluster the neighbourhoods

In [45]:
kl_mall.drop(["Neighborhoods"], 1)

,Shopping Mall
0,1
1,0
2,0
3,1
4,0
5,2
6,1
7,0
8,1
9,1


Run k-means to cluster the neighborhoods in Hyderabad into 3 clusters.

In [34]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 0, 2, 0, 1, 2, 0, 2, 2], dtype=int32)

In [35]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [36]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head(10)

,Neighborhood,Shopping Mall,Cluster Labels
0,A. C. Guards,1,2
1,A. S. Rao Nagar,0,0
2,Abhyudaya Nagar,0,0
3,Abids,1,2
4,Adikmet,0,0
5,Afzal Gunj,2,1
6,Aghapura,1,2
7,"Aliabad, Hyderabad",0,0
8,Alijah Kotla,1,2
9,Allwyn Colony,1,2


In [37]:
#Add latitude and longitude values by using the join operation(the new dataframe with the old dataframe containing the latitude and longitude values)
#kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

#print(kl_merged.shape)
kl_merged['Latitude'] = df_coords['Latitude']
kl_merged['Longitude'] = df_coords['Longitude']
kl_merged.head() # check the last columns!

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,A. C. Guards,1,2,17.395015,78.459812
1,A. S. Rao Nagar,0,0,17.411200,78.508240
2,Abhyudaya Nagar,0,0,17.337650,78.564140
3,Abids,1,2,17.389800,78.476580
4,Adikmet,0,0,17.235790,78.541300


In [38]:
# sorting the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(198, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
98,Kondapur,0,0,17.325980,78.447350
117,Madhapur,0,0,17.357880,78.501710
120,Mahatma Gandhi Road (Secunderabad),0,0,17.396090,78.405170
121,Malakpet,0,0,17.379280,78.477490
122,Malkajgiri,0,0,17.428497,78.487907
123,Malkajgiri mandal,0,0,17.374930,78.515670
124,Mallapur,0,0,17.447370,78.535200
126,Manikonda,0,0,17.288640,78.497960
127,Marredpally,0,0,17.391860,78.459050
129,Meerpet–Jillelguda,0,0,17.447790,78.508700


In [39]:
kl_merged['Shopping Mall'].max()

2

# Now we visualize the resulting clusters

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [41]:
# save the map as HTML file
map_clusters.save('./map_clusters.html')

# Examine clusters

Cluster 0

In [42]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]
#len(kl_merged.loc[kl_merged['Cluster Labels'] == 0])# -132 neighbourhoods/places in this cluster 0

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
98,Kondapur,0,0,17.325980,78.447350
117,Madhapur,0,0,17.357880,78.501710
120,Mahatma Gandhi Road (Secunderabad),0,0,17.396090,78.405170
121,Malakpet,0,0,17.379280,78.477490
122,Malkajgiri,0,0,17.428497,78.487907
123,Malkajgiri mandal,0,0,17.374930,78.515670
124,Mallapur,0,0,17.447370,78.535200
126,Manikonda,0,0,17.288640,78.497960
127,Marredpally,0,0,17.391860,78.459050
129,Meerpet–Jillelguda,0,0,17.447790,78.508700


Cluster 1

In [43]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]
#len(kl_merged.loc[kl_merged['Cluster Labels'] == 1])
#There are 51 neighbourhoods/places in cluster 1 which is the highest among the 3 clusters

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
5,Afzal Gunj,2,1,17.410610,78.515130
168,Punjagutta,2,1,17.394870,78.470760
26,Banjara Hills,2,1,17.415350,78.434350
136,Moazzam Jahi Market,2,1,17.366203,78.457983
13,Ameerpet,2,1,17.533320,78.325290
102,Kukatpally,2,1,17.385940,78.483380
119,Maharajgunj,2,1,17.456940,78.390130
174,Raj Bhavan Road,2,1,17.425110,78.395220


Cluster 2

In [44]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2] 
#len(kl_merged.loc[kl_merged['Cluster Labels'] == 2] ) #-There are only 9 shopping malls in the cluster which is the least among the 3 clusters

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
19,Azampura,1,2,17.407100,78.502330
181,"Red Hills, Hyderabad",1,2,17.429249,78.490431
11,Amberpet,1,2,17.535430,78.544270
183,Risala Bazar,1,2,17.355540,78.497560
21,Badichowdi,1,2,17.388960,78.486810
3,Abids,1,2,17.389800,78.476580
166,Pisal Banda,1,2,17.442320,78.496170
170,Putlibowli,1,2,17.426160,78.452100
176,Ramakrishnapuram,1,2,17.511610,78.294110
190,Sanghi Nagar,1,2,17.477175,78.528480


# Final observation

A good number of shopping malls are concentrated in the central area of Hyderabad city, with the highest number in cluster 2 and moderate number in cluster 1. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighbourhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighbourhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighbourhoods in cluster 2 which already have a high concentration of shopping malls and suffering from intense competition.




<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7d359fea-d5df-4ee4-ab57-c22bac6c0028' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>